In [1]:
import pandas as pd
import requests
import json
import ipywidgets as widgets

In [2]:
%load_ext jupyter_black

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
current = pd.read_csv(
    "https://theunitedstates.io/congress-legislators/legislators-current.csv"
)
historic = pd.read_csv(
    "https://theunitedstates.io/congress-legislators/legislators-historical.csv"
)
politicians = pd.concat([current, historic])
politicians.reset_index(
    drop=True, inplace=True
)  # Reset duplicate indexes after merging
politicians.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,twitter_id,facebook,youtube,youtube_id,mastodon,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,NaN,1.0,Democrat,https://www.brown.senate.gov,503 Hart Senate Office Building Washington DC ...,202-224-2315,https://www.brown.senate.gov/contact/,http://www.brown.senate.gov/rss/feeds/?type=al...,SenSherrodBrown,43910797.0,SenatorSherrodBrown,SherrodBrownOhio,UCgy8jfERh-t_ixkKKoCmglQ,NaN,B000944,136.0,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,NaN,1.0,Democrat,https://www.cantwell.senate.gov,511 Hart Senate Office Building Washington DC ...,202-224-3441,https://www.cantwell.senate.gov/public/index.c...,http://www.cantwell.senate.gov/public/index.cf...,SenatorCantwell,117501995.0,senatorcantwell,SenatorCantwell,UCN52UDqKgvHRk39ncySrIMw,NaN,C000127,172.0,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,NaN,1.0,Democrat,https://www.cardin.senate.gov,509 Hart Senate Office Building Washington DC ...,202-224-4524,https://www.cardin.senate.gov/contact/,http://www.cardin.senate.gov/rss/feeds/?type=all,SenatorCardin,109071031.0,senatorbencardin,senatorcardin,UCiQaJnMzlfzzG3VESgyZChA,NaN,C000141,174.0,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,NaN,1.0,Democrat,https://www.carper.senate.gov,513 Hart Senate Office Building Washington DC ...,202-224-2441,https://www.carper.senate.gov/contact/,http://www.carper.senate.gov/public/index.cfm/...,SenatorCarper,249787913.0,tomcarper,senatorcarper,UCgLnvbKwu4B3navofj6Qvvw,NaN,C000174,179.0,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,NaN,1.0,Democrat,https://www.casey.senate.gov,393 Russell Senate Office Building Washington ...,202-224-6324,https://www.casey.senate.gov/contact,http://www.casey.senate.gov/rss/feeds/?all,SenBobCasey,171598736.0,SenatorBobCasey,SenatorBobCasey,UCtVssXhx-KuZa-hSvnsnJ0A,NaN,C001070,1828.0,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.


In [ ]:
def defineNamePidMapping(df):
    print(
        "Mapping columns to PIDs and downloading Wikidata data. To skip column, leave blank."
    )
    colNamePid = {}
    for col in df.columns:
        first_non_null_index = df[col].first_valid_index()
        first_non_null_value = df[col][first_non_null_index]

        pid = input(f"PID for {col}: (example: {first_non_null_value}) ").upper()
        if len(pid) == 0:
            colNamePid[col] = None
            continue
        if pid[0] != "P":
            pid = "P" + pid
        colNamePid[col] = pid
    
    return colNamePid

with open("colNamePid.json", "w") as file:
    json.dump(defineNamePidMapping(politicians), file)

In [ ]:
with open("colNamePid.json", "r") as file:
    colNamePid = json.load(file)
colNamePid